# This NB demonstrates how we can use methods from visualizations

We can use the method to plot all patterns, double click on specific patterns to highlight them in the heatmaps, and select top patterns to plot them on their own.

In [1]:
# python imports
import pandas as pd
import altair as alt

# imports to load relationships
import requests
from pandas.io.json import json_normalize

# our imports
%load_ext autoreload
%autoreload 2
%cd ../intervals/
from main import *
%cd ../intervals/
import visualizations as viz

/Users/dangtrang/OneDrive - brynmawr.edu/summer 2021/crim_intervals/intervals
/Users/dangtrang/OneDrive - brynmawr.edu/summer 2021/crim_intervals/intervals


## `plot_ngrams_heatmap`

First, we do the normal steps to get a ngrams dataframe.

In [31]:
root = "https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_3.0/"
prefix = "CRIM_Model_00"
files = ["17"] 
postfix = ".mei"
corpus = CorpusBase([root + prefix + files[0] + postfix])
model = corpus.scores[0]
mel = model.getMelodic(kind='d', compound=False, unit=0)
mel_ngrams = model.getNgrams(df=mel, n=5)
mel_ngrams.head()

Memoized piece detected...


,[Discantus],[Contratenor],[Tenor],[QuintaVox],[SextaVox],[Bassus]
4.0,NaN,"1, 2, 2, 2, -4",NaN,NaN,NaN,NaN
7.0,NaN,"2, 2, 2, -4, 8",NaN,NaN,NaN,NaN
8.0,"1, 2, 1, 2, -3","2, 2, -4, 8, -2",NaN,NaN,NaN,NaN
10.0,NaN,"2, -4, 8, -2, -2",NaN,NaN,NaN,NaN
12.0,NaN,"-4, 8, -2, -2, 2",NaN,NaN,NaN,NaN


Can read the documentation with `help(viz.plot_ngrams_heatmap)`

In [32]:
help(viz.plot_ngrams_heatmap)

Help on function plot_ngrams_heatmap in module visualizations:

plot_ngrams_heatmap(ngrams_df, ngrams_duration=None, selected_patterns=[], voices=[], heatmap_width=800, heatmap_height=300)
    Plot a heatmap for crim-intervals getNgram's output.
    :param ngrams_df: crim-intervals getNgram's output
    :param ngrams_duration: if not None, rely on durations in the
    df to calculate the durations of the ngrams.
    :param selected_patterns: list of specific patterns the users want (optional)
    :param voices: list of specific voices the users want (optional)
    :param heatmap_width: the width of the final heatmap (optional)
    :param heatmap_height: the height of the final heatmap (optional)
    :return: a bar chart that displays the different patterns and their counts,
    and a heatmap with the start offsets of chosen voices / patterns



In [33]:
viz.plot_ngrams_heatmap(mel_ngrams,selected_patterns=[], voices=[])

alt.VConcatChart(...)

### Selecting patterns

We could only include some patterns in the heatmaps

First, I collected the top patterns and turned these patterns into a list.

In [34]:
# count and get the 10 most popular patterns
mel_ngrams_top_patterns_df = mel_ngrams.stack().value_counts().to_frame().head(10)
# retrieve a list to generate heatmaps
mel_ngrams_top_patterns_list = mel_ngrams_top_patterns_df.index.to_list()
mel_ngrams_top_patterns_df

,0
"-2, -2, -2, -2, -2",21
"2, 2, 2, 2, 2",20
"2, 2, 2, 2, -2",16
"2, 2, 2, 2, -3",14
"2, 2, 2, -3, 2",13
"2, 2, -2, -3, 2",13
"2, -2, -3, 2, -2",11
"-2, 2, 2, 2, 2",11
"-2, -3, 2, 2, 2",10
"1, 2, 2, -2, -3",10


In [35]:
mel_ngrams_top_patterns_list 

['-2, -2, -2, -2, -2',
 '2, 2, 2, 2, 2',
 '2, 2, 2, 2, -2',
 '2, 2, 2, 2, -3',
 '2, 2, 2, -3, 2',
 '2, 2, -2, -3, 2',
 '2, -2, -3, 2, -2',
 '-2, 2, 2, 2, 2',
 '-2, -3, 2, 2, 2',
 '1, 2, 2, -2, -3']

We would pass the list of patterns into the `patterns` parametter.

In [36]:
viz.plot_ngrams_heatmap(mel_ngrams, selected_patterns=mel_ngrams_top_patterns_list)

alt.VConcatChart(...)

**Cross-checking output from the heatmap with the ngrams dataframe**

We can see that'1, 1, 1, 1, 1' is a really popular pattern. Let's view its start and end points in ngram and in the heatmap's dataframe to make sure that it has been calculated correctly

## `generate_ngrams_and_dur` allow us to create heatmap with durations!

How could we create a heatmap for ngrams in which the ngrams' durations are desplayed?

In [40]:
ngrams, ngrams_duration = viz.generate_ngrams_and_dur(model, mel, n=5)

In [42]:
help(viz.generate_ngrams_and_dur)

Help on function generate_ngrams_and_dur in module visualizations:

generate_ngrams_and_dur(model, df, n=3, exclude=['Rest'], interval_settings=('d', True, True), offsets='first')
    This method accept a model and a dataframe with the melody or notes
    and rests and generate an ngram (in columnwise and unit=0 setting)
    and a corresponding duration ngram
    :param model: an Imported Piece object.
    :param df: dataframe containing consecutive notes.
    :param n: (refer to getNgrams documentation)
    :param exclude: (refer to getNgrams documentation)
    :param interval_settings: (refer to getNgrams documentation)
    :param offsets: (refer to getNgrams documentation)
    :return: ngram and corresponding duration dataframe!



In [41]:
viz.plot_ngrams_heatmap(ngrams, ngrams_duration, selected_patterns=mel_ngrams_top_patterns_list , voices=[])

alt.VConcatChart(...)

## `plot_close_match_heatmap`

In [14]:
help(viz.plot_close_match_heatmap)

Help on function plot_close_match_heatmap in module visualizations:

plot_close_match_heatmap(ngrams_df, key_pattern, ngrams_duration=None, selected_patterns=[], voices=[], heatmap_width=800, heatmap_height=300)
    Plot how closely the other vectors match a selected vector.
    Uses the Levenshtein distance.
    :param ngrams_df: crim-intervals getNgram's output
    :param key_pattern: a pattern the users selected to compare other patterns with (tuple of floats)
    :param selected_pattern: the specific other vectors the users selected
    :param ngrams_duration: if None, simply output the offsets. If the users input a
    list of durations, caculate the end by adding durations with offsets and
    display the end on the heatmap accordingly.
    :param selected_patterns: list of specific patterns the users want (optional)
    :param voices: list of specific voices the users want (optional)
    :param heatmap_width: the width of the final heatmap (optional)
    :param heatmap_height: t

In [43]:
viz.plot_close_match_heatmap(mel_ngrams, '-2, -2, -2, -2, -2', selected_patterns=[], voices=[])

alt.Chart(...)

In [44]:
# use the durations calculated from above
viz.plot_close_match_heatmap(ngrams,'-2, -2, -2, -2, -2', ngrams_duration, selected_patterns=[], voices=[])

alt.Chart(...)

## `plot_comparison_heatmap`

This method can be used to plot a relationship/observation json

### For relationships

In [17]:
# data_relationships = requests.get('http://crimproject.org/data/relationships/').json()
# df_relationships = pd.json_normalize(data_relationships)
df_relationships = pd.read_csv('../CRIM_Intervals_Notebooks/relationships.csv')
files = df_relationships['model_observation.piece.piece_id'].unique()
df_relationships_test = df_relationships[df_relationships['model_observation.piece.piece_id'] == files[1]].copy()

In [18]:
#df_relationships.to_csv('../CRIM_Intervals_Notebooks/relationships.csv')
df_relationships_test.head()

,Unnamed: 0,url,id,relationship_type,musical_type,rt_q,rt_q_x,rt_q_monnayage,rt_tm,rt_tm_snd,...,model_observation.piece.url,model_observation.piece.piece_id,model_observation.piece.full_title,model_observation.ema,derivative_observation.url,derivative_observation.id,derivative_observation.piece.url,derivative_observation.piece.piece_id,derivative_observation.piece.full_title,derivative_observation.ema
4,4,http://crimproject.org/data/relationships/5/,5,Quotation,Soggetto,True,True,False,False,False,...,http://crimproject.org/data/pieces/CRIM_Model_...,CRIM_Model_0018,Baisez moy,"1-38/6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6...",http://crimproject.org/data/observations/10/,10,http://crimproject.org/data/pieces/CRIM_Mass_0...,CRIM_Mass_0017_4,Missa Baisez moy: Sanctus,"1-41/5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5..."
5,5,http://crimproject.org/data/relationships/6/,6,Mechanical transformation,Fuga,False,False,False,True,True,...,http://crimproject.org/data/pieces/CRIM_Model_...,CRIM_Model_0018,Baisez moy,"1-38/3+6,3+6,3+6,3+6,3+6,3+6,3+6,3+6,3+6,3+6,3...",http://crimproject.org/data/observations/12/,12,http://crimproject.org/data/pieces/CRIM_Mass_0...,CRIM_Mass_0017_4,Missa Baisez moy: Sanctus,"1-41/5,5,5,5,4-5,4-5,4-5,4-5,4-5,4-5,4-5,4-5,4..."
6,6,http://crimproject.org/data/relationships/7/,7,Non-mechanical transformation,Fuga,False,False,False,False,False,...,http://crimproject.org/data/pieces/CRIM_Model_...,CRIM_Model_0018,Baisez moy,"1-4/2,1-2,1-2,1/@3,@3+@1-3,@1-3+@1,@1",http://crimproject.org/data/observations/14/,14,http://crimproject.org/data/pieces/CRIM_Mass_0...,CRIM_Mass_0017_4,Missa Baisez moy: Sanctus,"1-10/1-2,1-2,1-2,1-2,1-2,1-2,1-2,1,1,1/@all+@1..."
7,7,http://crimproject.org/data/relationships/8/,8,Mechanical transformation,Fuga,False,False,False,True,False,...,http://crimproject.org/data/pieces/CRIM_Model_...,CRIM_Model_0018,Baisez moy,"1-5/1-2,1-2,1-2,1-2,1/@all+@3,@1-3+@1-3,@1-3+@...",http://crimproject.org/data/observations/16/,16,http://crimproject.org/data/pieces/CRIM_Mass_0...,CRIM_Mass_0017_5,Missa Baisez moy: Agnus Dei,"42-47/2,2,2,1-2,1,1/@1,@1-3,@1-4.5,@1+@1,@1-3,@1"
548,548,http://crimproject.org/data/relationships/557/,557,Quotation,NaN,True,True,False,False,False,...,http://crimproject.org/data/pieces/CRIM_Model_...,CRIM_Model_0018,Baisez moy,3/1/@2,http://crimproject.org/data/observations/1114/,1114,http://crimproject.org/data/pieces/CRIM_Model_...,CRIM_Model_0011,Tota pulchra es,5/2/@2.5


In [19]:
help(viz.plot_comparison_heatmap)

Help on function plot_comparison_heatmap in module visualizations:

plot_comparison_heatmap(df, ema_col, main_category='musical_type', other_category='observer.name', option=1, heat_map_width=800, heat_map_height=300)
    This method plots a chart for relationships/observations dataframe retrieved from their
    corresponding json files. This chart has two bar charts displaying the count of variables
    the users selected, and a heatmap displaying the locations of the relationship.
    :param df: relationships or observations dataframe
    :param ema_col: name of the ema column
    :param main_category: name of the main category for the first bar chart.
    The chart would be colored accordingly (default='musical_type').
    :param other_category: name of the other category for the zeroth bar chart.
    (default='observer.name')
    :param heat_map_width: the width of the final heatmap (default=800)
    :param heat_map_height: the height of the final heatmap (default =300)
    :return

In [20]:
viz.plot_comparison_heatmap(df_relationships_test, 'model_observation.ema', main_category='musical_type', other_category='observer.name', heat_map_width=800,
                              heat_map_height=300)

alt.VConcatChart(...)

### For observations

In [21]:
# data_observations = requests.get('http://crimproject.org/data/observations/').json()
# df_observations = pd.json_normalize(data_observations)
# df_observations.to_csv('../CRIM_Intervals_Notebooks/observations.csv')

In [22]:
df_observations = pd.read_csv('../CRIM_Intervals_Notebooks/observations.csv')
df_observations_test = df_observations[df_observations['piece.piece_id'] == 'CRIM_Model_0011'].copy()
df_observations_test.head()

,Unnamed: 0,url,id,ema,musical_type,mt_cf,mt_cf_voices,mt_cf_dur,mt_cf_mel,mt_sog,...,remarks,created,updated,curated,observer.url,observer.name,piece.url,piece.piece_id,piece.full_title,piece.mass
0,0,http://crimproject.org/data/observations/1/,1,"1-6/1,1,1-2,1-2,2,2/@1,@1-3,@1-3+@1,@1-3+@1-3,...",Fuga,False,NaN,False,False,False,...,Makes up a longer ID,2017-06-19T19:22:54-04:00,2020-11-27T15:11:10.011852-05:00,True,http://crimproject.org/data/people/CRIM_Person...,Ian Lorenz,http://crimproject.org/data/pieces/CRIM_Model_...,CRIM_Model_0011,Tota pulchra es,NaN
2,2,http://crimproject.org/data/observations/3/,3,"5-9/1,1,1-2,2,2/@3,@1-4,@1-2+@4,@1-4,@1-2",Fuga,False,NaN,False,False,False,...,NaN,2017-06-19T19:31:29-04:00,2020-08-02T16:14:06.430415-04:00,True,http://crimproject.org/data/people/CRIM_Person...,Ian Lorenz,http://crimproject.org/data/pieces/CRIM_Model_...,CRIM_Model_0011,Tota pulchra es,NaN
4,4,http://crimproject.org/data/observations/5/,5,"5-9,14-18/1,1,1-2,1-2,1-2,3,3,3-4,3-4,3-4/@3,@...",Fuga,False,NaN,False,False,False,...,NaN,2017-06-19T20:36:44-04:00,2020-08-02T16:19:38.187899-04:00,True,http://crimproject.org/data/people/CRIM_Person...,Drew Ivarson,http://crimproject.org/data/pieces/CRIM_Model_...,CRIM_Model_0011,Tota pulchra es,NaN
6,6,http://crimproject.org/data/observations/7/,7,"5-9/1,1,1-2,2,2/@3,@1-4,@1-2+@4,@1-4,@1-2",Fuga,False,NaN,False,False,False,...,NaN,2017-06-20T12:18:12-04:00,2019-06-26T17:55:17.044000-04:00,True,http://crimproject.org/data/people/CRIM_Person...,David Fiala,http://crimproject.org/data/pieces/CRIM_Model_...,CRIM_Model_0011,Tota pulchra es,NaN
366,366,http://crimproject.org/data/observations/367/,367,"1-7,10-16/1,1,1-2,1-2,2,2,2,3,3,3-4,3-4,4,4,4/...",Imitative duo,False,NaN,False,False,False,...,NaN,2017-07-11T20:33:14-04:00,2020-12-20T17:29:27.130591-05:00,True,http://crimproject.org/data/people/CRIM_Person...,Manon Manfredi,http://crimproject.org/data/pieces/CRIM_Model_...,CRIM_Model_0011,Tota pulchra es,NaN


In [23]:
viz.plot_comparison_heatmap(df_observations_test, 'ema', main_category='musical_type', other_category='observer.name', heat_map_width=800,
                              heat_map_height=300)

alt.VConcatChart(...)

## Generate networks with `create_comparisons_networks_and_interactive_df`

We can create a some network visualizations and a df to interact with patterns.

Work with both time interval and melodic interval!

**Important**:
- The plot will be saved in a html file in the folder you choose.
- The plot might not be displayed if it's not saved in the same folder as the Jupyter Notebook, but you should be able to find it and open it in your computer if necessary.

### time interval for pens

The columns of interest from table containing observations fetched from the CRIM database would be `mt_pe_tint` (mt= mechanical transformation, pe= periodic entries, tint=time intervals) and we would choose `interval_type` to be `time`. The table's corresponding ema column is `ema`.

In [56]:
pen_networks, pen_widget = viz.create_comparisons_networks_and_interactive_df(df_observations, 'mt_pe_tint', 'time', 'ema')

interactive(children=(Text(value='', description='search_pattern_starts_with'), Output()), _dom_classes=('widg…

In [50]:
# see all the avialable patterns
pen_networks

{'all': <class 'pyvis.network.Network'> |N|=64 |E|=378,
 'nan': <class 'pyvis.network.Network'> |N|=0 |E|=0,
 'B1': <class 'pyvis.network.Network'> |N|=4 |E|=8,
 'S1': <class 'pyvis.network.Network'> |N|=15 |E|=33,
 'S2': <class 'pyvis.network.Network'> |N|=2 |E|=2,
 'M1': <class 'pyvis.network.Network'> |N|=3 |E|=5,
 'S3': <class 'pyvis.network.Network'> |N|=2 |E|=5,
 'M8': <class 'pyvis.network.Network'> |N|=3 |E|=10,
 'M6': <class 'pyvis.network.Network'> |N|=2 |E|=4,
 'M2': <class 'pyvis.network.Network'> |N|=4 |E|=4,
 'M4': <class 'pyvis.network.Network'> |N|=6 |E|=14,
 'M3': <class 'pyvis.network.Network'> |N|=7 |E|=6,
 'S4': <class 'pyvis.network.Network'> |N|=3 |E|=27,
 'S6': <class 'pyvis.network.Network'> |N|=2 |E|=1,
 'B2': <class 'pyvis.network.Network'> |N|=2 |E|=22,
 'M5': <class 'pyvis.network.Network'> |N|=1 |E|=0,
 'B3': <class 'pyvis.network.Network'> |N|=1 |E|=0,
 'B4': <class 'pyvis.network.Network'> |N|=2 |E|=1,
 'L1': <class 'pyvis.network.Network'> |N|=2 |E|=13,


In [51]:
# show all the patterns mapped onto one network
pen_networks['all'].show('../CRIM_Intervals_Notebooks/all_time_pen.html')

In [52]:
# show only one family of patterns mapped onto one network
pen_networks['S1'].show('../CRIM_Intervals_Notebooks/B1_time_pen.html')

### melodic intervals for fuga

The columns of interest from table containing observations fetched from the CRIM database would be `mt_fg_int` (mt= mechanical transformation, fg= Fuga, int=intervals) and we would choose `interval_type` to be `melodic`. The table's corresponding ema column is `ema`.

In [58]:
fuga_networks, fuga_widget = viz.create_comparisons_networks_and_interactive_df(df_observations, 'mt_fg_int', 'melodic', 'ema')

interactive(children=(Text(value='', description='search_pattern_starts_with'), Output()), _dom_classes=('widg…

In [61]:
# we are interested in the patterns that start with 4-
fuga_networks['4-'].show('../CRIM_Intervals_Notebooks/4-_fuga.html')